# Ingesta de datos desde S3

In [0]:
import pandas as pd
import boto3
import csv
import re
from io import StringIO

# Credenciales
ACCESS_KEY = "ACCESS_KEY"
SECRET_KEY = "SECRET_KEY"
BUCKET = "evargas-hackaton-databricks"
FOLDER = "Fuentes/"
archivos = ["Compras.csv", "Clientes.csv"]

s3_client = boto3.client('s3', aws_access_key_id=ACCESS_KEY, aws_secret_access_key=SECRET_KEY)

def limpiar_columnas(df):
    # Reemplaza espacios por guiones bajos, quita tildes y caracteres raros
    cols = df.columns
    cols = [c.lower().replace(' ', '_').replace('.', '') for c in cols]
    cols = [re.sub(r'[áéíóú]', lambda x: {'á':'a','é':'e','í':'i','ó':'o','ú':'u'}[x.group()], c) for c in cols]
    df.columns = cols
    return df

for nombre in archivos:
    try:
        obj = s3_client.get_object(Bucket=BUCKET, Key=FOLDER + nombre)
        contenido = obj['Body'].read().decode('latin-1')
        
        df_pd = pd.read_csv(
            StringIO(contenido), 
            sep=None, 
            engine='python', 
            on_bad_lines='skip',
            quoting=csv.QUOTE_MINIMAL
        )
        
        # LIMPIEZA DE COLUMNAS PARA DELTA
        df_pd = limpiar_columnas(df_pd)
        
        # Convertir a Spark
        df_spark = spark.createDataFrame(df_pd.astype(str))
        
        # Guardar en Bronze
        nombre_tabla = f"workspace.bronze.{nombre.replace('.csv', '').lower()}"
        df_spark.write.mode("overwrite").option("overwriteSchema", "true").saveAsTable(nombre_tabla)
        print(f"✅ {nombre} cargado exitosamente en {nombre_tabla}")
        
    except Exception as e:
        print(f"❌ Error crítico en {nombre}: {e}")